In [1]:

import sel_scrape as sc
import pandas as pd
import sys,os
import datetime
import urllib.request as ur
import importlib
import datetime
import time
import pdb
import pathlib
MY_HOME = str(pathlib.Path.home())

In [ ]:
pd.set_option('display.max_colwidth', -1)
PROFILE_PATH = f'{MY_HOME}/Library/Application Support/Google/Chrome/Default'

### Scrap Peapod for new delivery times
1. Fill in a date for the variable ```month_dd```
2. Run the cells below

In [ ]:
month_dd = "April 25"
wait_time_between_loops = 5*60 # 5 minutes

### Define import methods to click and search

In [ ]:
def dismiss_modal(sac):
    modal_close = "//span[@class='optly-modal-close']"
    el = sac.findxpath(modal_close)
    if el is not None and el['status'] is None:
        sac.click_element(modal_close)


def click_and_wait(d,sac,wait_time=2):
    for _ in range(2):
        el = sac.findxpath(d)    
        if el is not None and el['status'] is None:
            c1 = sac.click_element(d)
            return c1
        time.sleep(wait_time)
    return None        

def click_pickup_times(sac,wait_time=2):
    d= "//a[@aria-label='footer delivery / pick-up times']"
    return click_and_wait(d,sac,wait_time=wait_time)

def click_edit_times(sac,wait_time=2):
    d= "//button[contains(@class,'button--prime button-width--med')]"
    return click_and_wait(d,sac,wait_time=wait_time)

def click_continue_shopping(sac,wait_time=2):
    d = "//button[@id='slot-continue-shopping-button']"
    return click_and_wait(d,sac,wait_time=wait_time)

def find_peapod(sac,month_dd):
    timexp = f"//li[contains(@aria-label,'{month_dd}')]"

    el=None
    found = False
    try:
        el = sac.findxpath(timexp)
    except:
        pass
    if el is not None:
        if el['status'] is None:
            print(datetime.datetime.now())
            print(el['value'][0].text)
            found=True
    return found

### Step 01: create a browser window

In [ ]:
pp=PROFILE_PATH
sac = sc.SelScrape(headless=False,driver_name='chrome',
                  profile_path=pp)
#                    executable_path='./drivers/geckodriver')

In [ ]:
sac.goto('https://www.peapod.com/')
time.sleep(2)
print('try to dismiss modal')
dismiss_modal(sac)

input("Make sure that you are logged ub to Peapod, hit enter to continue, hit the stop buttom to stop: ")



### Scan for new peapod month and year

In [ ]:
found=False
while not found:
    print("click pickup times")
    click_pickup_times(sac)
    time.sleep(2)
    print("click edit times")
    click_edit_times(sac)
    time.sleep(2)
    print("find date")
    found = find_peapod(sac,month_dd)
    if not found:
        time.sleep(2)
        print('continue_shopping')
        click_continue_shopping(sac)
        time.sleep(wait_time_between_loops)
    else:
        print(f'found date at {datetime.datetime.now()}')
        for _ in range(30):
            os.system("echo -ne '\007'")
            time.sleep(.5)
        input('waiting to attempt to get delivery: hit return when finished')
        